__качаем объявления о продаже квартир с циан.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
import sys
import re
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep
from random import random

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

In [3]:
raw_data_path = 'data/raw/flat/cian' # каталог с файлами объявлений

In [4]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'
# base_url, req_param = 'https://sevastopol.cian.ru/kupit-kvartiru/', ''
place = 'sevastopol'
base_url, req_param ='https://www.cian.ru/cat.php', 'deal_type=sale&offer_type=flat&region=184723'

In [5]:
%%time

# from lib.downloader import DownloaderSimple
# from lib.parser import AdsListParser
from lib.downloader import DownloaderSeleniumFirefox
from lib.cian import CianParser


df,html = CianParser(
        base_url = base_url,
        driver=DownloaderSeleniumFirefox(profile_path),
        # driver=DownloaderSimple(),
    ).load(req_param=req_param, keep_html=True) # , npage_start=53) # , page_limit=3 ) #

print(len(df))

[INFO    ] 2022-09-26 11:14:31 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2022-09-26 11:14:31 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2022-09-26 11:14:33 | AdsListParser: downloader init
[INFO    ] 2022-09-26 11:14:33 | CianParser: downloader init
[INFO    ] 2022-09-26 11:14:33 | AdsListParser: start read and parse pages...
[INFO    ] 2022-09-26 11:14:42 | AdsListParser: read page 1
[INFO    ] 2022-09-26 11:14:53 | AdsListParser: read page 2
[INFO    ] 2022-09-26 11:15:04 | AdsListParser: read page 3
[INFO    ] 2022-09-26 11:15:12 | AdsListParser: read page 4
[INFO    ] 2022-09-26 11:15:20 | AdsListParser: read page 5
[INFO    ] 2022-09-26 11:15:29 | AdsListParser: read page 6
[INFO    ] 2022-09-26 11:15:36 | AdsListParser: read page 7
[INFO    ] 2022-09-26 11:15:46 | AdsListParser: read page 8
[INFO    ] 2022-09-26 11:15:57 | AdsListParser: read page 9
[INFO    ] 2022-09-26 11:16:04 | AdsListParser: read page 10
[INFO    ] 2022-09-26 11:16:12 | AdsLis

In [6]:
df.sample(3)

,OfferTitle,OfferSubtitle,Deadline,MainPrice,PriceInfo,GeoLabel,TimeLabel,LinkArea,Description,page,ts
978,"1-комн. кв., 35,7 м², 5/11 этаж",,,6 000 000 ₽,168 067 ₽/м²,"[Севастополь, р-н Нахимовский, улица Горпищенко, 127к2]","[вчера, 24 сен, 21:07]",https://sevastopol.cian.ru/sale/flat/276480610/,"Продаётся 1кк видовая квартира в новом ЖК ""Атмосфера"" Площадь квартиры с балконом 40.5 квадратных метров. Балкон застеклён, можно использовать , как дополнительную комнату, размер балкона позволяет. В жилой комнате установлено большое панорамное окно. Окна квартиры выходят на Северо-Запад, а вид открывается потрясающий: на море , лес, город. Видно , как плывут корабли, виден Владимирский собор на Херсонесе. Жилая комната размером 19 кв.м, кухня 8,5 кв.м. Установлен газовый двухконтурный котёл, что позволяет экономить средства на коммунальных платежах и самостоятельно регулировать температуру в помещении, бесперебойно пользоваться горячей водой. У дома организована огромная стоянка для автомобилей, детская площадка. Выезд из города по новой асфальтированной дороге, без пробок. Школа - в 7 минутах на авто или 18 минут пешком. Детский сад - в 4 минутах на авто или 15 минут пешком. Остановка общественного транспорта - в 5-7 минутах ходьбы: ходит около 10-ти различных маршрутов. Магазины, кафе, кулинария, мастерские, СДЭК, Вайлдберриз располагаются как в цокольном этаже дома, так и в пешей доступности в соседних домах и кварталах. Возможна ипотека. Клиентам компании Этажи ипотечная консультация бесплатно. А так же помощь в одобрении ипотеки с ежемесячным платежом от 38 до 43 тыс.руб. Хотите пить кофе и любоваться как солнце садиться за море, скорее звоните! Оперативно организую показ. Покупателю с наличкой торг.\n . Номер в базе: 8003583.",35,2022-09-26 11:23:56.441542
1392,"3-комн. кв., 71,3 м², 5/5 этаж",,,9 000 000 ₽,126 227 ₽/м²,"[Севастополь, р-н Ленинский, мкр. Острякова, проспект Генерала Острякова, 196]","[вчера, 24 сен, 12:55]",https://sevastopol.cian.ru/sale/flat/277464407/,"Трехкомнатная квартира 71,3 м2, на 5/5 этаже дома с просторной кухней 11 м2.. Расположена в г. Севастополь, Ленинский район, пр-кт Генерала Острякова, д. 196.\nПозвоните нам, чтобы узнать все детали! \nТехнические характеристики: \n \n-Адрес: г. Севастополь, Ленинский район, пр-кт Генерала Острякова, д. 196\n-Год постройки дома: 1982\n-Этаж/этажность: 5/5\n-Количество комнат: 3\n-Комнаты: изолированные\n-Общая площадь: 71,3 м2\n-Жилая площадь: 40,4 м2\n-Комната: 18; 11,6 и 10,8 м2\n-Площадь кухни: 11 м2\n-Высота потолка: 2,8 м\n-Санузел: раздельный\n-Водоснабжение: центральное\n-Горячее водоснабжение: бойлер\n-Отопление: центральное\nДополнительная информация: \nУютная и просторная трёхкомнатная квартира ""Чешского проекта"" в одном из самых лучших районов города, в ленинском! Дом построен в 1982 году.\nОбщая площадь 71,3 м², комнаты 18; 11,6 и 10,8 м², кухня 11 м², санузел раздельный, имеются две лоджии, поменяны трубы водоснабжения, заменены радиаторы отопления.\nКвартира теплая и светлая, в отличном состоянии, 2 кондиционера.\nДвор большой, зеленый и тихий.\nВ подъезде есть место для хранения колясок и велосипедов.\nПозвоните нам, чтобы назначить просмотр! \nИнфраструктура: \nРядом с домом остановка общественного транспорта 38 школа.\nРазвитая инфраструктура, отличная транспортная развязка.\nВ шаговой доступности детсад 127, школы 38, 48; магазины Новус, ПУД, до рынка ""5 км"" и ТЦ SeaMall 3 остановки, до центра города 15 минут на транспорте.\nНесколько отделений банков, почта, детский больничный комплекс, кафе и фитнес- центры.\nПодходит для наличного расчета, под гражданскую ипотеку, материнский капитал, жилищные сертификаты. \nДокументы РФ проверены юристами и готовы к сделке. \nПрофессиональное сопровождение до получения права собственности.\nДобавьте предложение в закладки, чтобы не потерять!",50,2022-09-26 11:23:56.441542
285,Продажа от собственника,"3-комн. кв., 82 м², 12/16 этаж",,11 850 000 ₽,144 512 ₽/м²,"[Севастополь, р-н Гагаринский, мкр. 5

In [7]:
print(len(df))
df = pd.DataFrame(df).dropna().drop_duplicates(['LinkArea','Description',]).reset_index(drop=True)
print(len(df))

1509
1475


In [8]:
df['place']=place

In [9]:
ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
assert len(df)>0
# df.to_excel(f'{raw_data_path}/cian_{ts}_raw.xlsx',index=False)
df.to_pickle(f'{raw_data_path}/cian_{ts}_raw.pkl')

----

In [10]:
# df = pd.read_excel('data/raw/cian/cian_2022-09-08_13-49_raw.xlsx')

In [11]:
# df

In [12]:
# df['GeoLabel'].apply( lambda s : len(s) ) #', '.join(s) )

# s = ['Севастополь', 'р-н Гагаринский', 'Казачья Бухта мкр', 'Скифия ЖК']
# ', '.join(s)

In [13]:
# from lib.cian import CianDataCleaner

# CianDataCleaner().transform(df)

----

In [14]:
# with open('tmp/cian.html','wt') as f: f.write(html[0])